<a href="https://colab.research.google.com/github/aldnoahh/plant-disease-recognition/blob/master/PDR_ResNet152V2_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Plant Disease Recognition using ResNet152V2 on modified version of PlantVillage Dataset.

Importing necessary libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications.resnet_v2 import ResNet152V2 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

Downloading and unzipping the modified dataset available on Gdrive. If you don`t have gdown module, install it using pip.

In [ ]:
#capture command here suppresses the large output
%%capture
!gdown --id 1Mj6wsKBZN2ycAyyIMs2lI361deuCJqBI --output pv0.zip
!unzip pv0.zip

Check if the folder has been unzipped.

In [ ]:
!ls

Setting up path for datagenerators from keras

In [ ]:
train_path = '/content/pv0/train'
valid_path = '/content/pv0/test'
# useful for getting number of files
image_files = glob(train_path + '/*/*.JPG')
valid_image_files = glob(valid_path + '/*/*.JPG')
# useful for getting number of classes
folders = glob(train_path + '/*')
len(folders)

Specify input image size.

In [ ]:
IMAGE_SIZE = [256, 256]

In [ ]:
#sneek peek at a random image
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

Configuring the pretrainned model as per our needs.

In [ ]:
ptm = PretrainedModel(
    input_shape=IMAGE_SIZE + [3],
    weights='imagenet',
    include_top=False)
# freeze pretrained model weights
ptm.trainable = False

In [ ]:
K = len(folders) # number of classes

#model definition
x = Flatten()(ptm.output)
x= BatchNormalization()(x)
x= Dense(512,activation='relu')(x)
x = Dense(K, activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=ptm.input, outputs=x)

In [ ]:
# view the structure of the model
model.summary()

In [ ]:
#view the number of layers in the model
len(model.layers)

In [ ]:
# create an instance of ImageDataGenerator
#Keras generators returns one-hot encoded labels and provides data augmentation.
gen_train = ImageDataGenerator(
  rotation_range=90,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  preprocessing_function=preprocess_input
)

gen_test = ImageDataGenerator(
  preprocessing_function=preprocess_input
)

In [ ]:
#batch size is the number of examples that are run through the model at once.
batch_size = 300

# create generators
train_generator = gen_train.flow_from_directory(
  train_path,
  shuffle=True,
  target_size=IMAGE_SIZE,
  batch_size=batch_size,
)
valid_generator = gen_test.flow_from_directory(
  valid_path,
  target_size=IMAGE_SIZE,
  batch_size=batch_size,
)

Since Keras no longer provides some metrics within itself, so we define those metrics ourselves. Here, we are defining F1_score, Precision and Recall.

In [ ]:
from keras import backend as Ke

def recall_m(y_true, y_pred):
    true_positives = Ke.sum(Ke.round(Ke.clip(y_true * y_pred, 0, 1)))
    possible_positives = Ke.sum(Ke.round(Ke.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + Ke.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = Ke.sum(Ke.round(Ke.clip(y_true * y_pred, 0, 1)))
    predicted_positives = Ke.sum(Ke.round(Ke.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Ke.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+Ke.epsilon()))

This block is for creating a lr scheduler, since the lr scheduler was not as effective as using adam directly, it is left for experimentation.

In [ ]:
# from keras.optimizers import SGD
# import math
# def step_decay(epoch):
# 	initial_lrate = 1e-4
# 	drop = 0.5
# 	epochs_drop = 10.0
# 	lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
# 	return lrate
# sgd = SGD(lr=0.0, momentum=0.9)
# # learning schedule callback
# from keras.callbacks import LearningRateScheduler
# lrate = LearningRateScheduler(step_decay)
# callbacks_list = [lrate]

Compiling our model with loss, optimizer and metrics (including our custom defined ones).

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy',f1_m,precision_m, recall_m]
)

The fit function is called for starting our training. 


In [ ]:
# fit the model
r = model.fit(
  train_generator,
  validation_data=valid_generator,
  epochs=5,
  steps_per_epoch=int(np.ceil(len(image_files) / batch_size)),
  validation_steps=int(np.ceil(len(valid_image_files) / batch_size)),
)

Saving our Model in HD5 format.

In [ ]:
model.save("model.h5")
print("Saved model to disk")

Graphs for our metrics

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# f1_score
plt.plot(r.history['f1_m'], label='train f1_m')
plt.plot(r.history['val_f1_m'], label='val f1_m')
plt.legend()
plt.show()

In [ ]:
# precision
plt.plot(r.history['precision_m'], label='train precision_m')
plt.plot(r.history['val_precision_m'], label='val precision_m')
plt.legend()
plt.show()

In [ ]:
# recall
plt.plot(r.history['recall_m'], label='train recall_m')
plt.plot(r.history['val_recall_m'], label='val recall_m')
plt.legend()
plt.show()

Next we evaluate the model on our test set again.

In [ ]:
#Load saved model from training
from keras.models import load_model
amod= load_model('/content/model.h5')
# evaluate the model
valid_generator = gen_test.flow_from_directory(valid_path,target_size=IMAGE_SIZE,batch_size=batch_size,)
loss, accuracy, f1_score, precision, recall = amod.evaluate(valid_generator, steps=int(np.ceil(len(valid_image_files)/ batch_size)))

Printing our metrics

In [ ]:
print('loss     : ',loss)
print('accuracy : ',accuracy)
print('f1_score :',f1_score)
print('precision:',precision)
print('recall   :',recall)